In [2]:
import pyspark
from pyspark.sql import SparkSession
import findspark
import pandas as pd

In [3]:
findspark.init()

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [6]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [7]:
df_pandas = pd.read_csv('fhvhv_tripdata_2021-01.csv.gz', nrows = 1000)

df_pandas

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,NaN
1,HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,NaN
2,HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,NaN
3,HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,NaN
4,HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,NaN
...,...,...,...,...,...,...,...
995,HV0003,B02871,2021-01-01 00:33:43,2021-01-01 00:47:52,255,61,NaN
996,HV0005,B02510,2021-01-01 00:53:10,2021-01-01 01:21:09,114,21,NaN
997,HV0003,B02512,2021-01-01 00:18:35,2021-01-01 00:26:53,167,42,NaN
998,HV0003,B02512,2021-01-01 00:35:39,2021-01-01 00:42:59,116,116,NaN


In [8]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [30]:
from pyspark.sql import types

In [9]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [10]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [11]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [12]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('SR_Flag', 'string')]

In [13]:
df = df.repartition(24)

In [14]:
df.write.parquet('fhvhv/2021/01')

In [15]:
df = spark.read.parquet('fhvhv/2021/01')

In [17]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-30 04:13:09|2021-01-30 04:26:02|         261|          50|   NULL|
|           HV0003|              B02871|2021-01-14 08:25:44|2021-01-14 08:35:06|           4|          88|   NULL|
|           HV0003|              B02883|2021-01-30 23:31:39|2021-01-30 23:45:53|          78|          81|   NULL|
|           HV0005|              B02510|2021-01-16 17:51:42|2021-01-16 18:05:49|         213|          58|   NULL|
|           HV0003|              B02882|2021-01-23 15:33:08|2021-01-23 15:53:39|         188|         188|   NULL|
|           HV0005|              B02510|2021-01-04 15:08:11|2021-01-04 15:18:02|

In [18]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [22]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-14 08:25:44|2021-01-14 08:35:06|           4|          88|
|2021-01-30 23:31:39|2021-01-30 23:45:53|          78|          81|
|2021-01-23 15:33:08|2021-01-23 15:53:39|         188|         188|
|2021-01-24 14:26:54|2021-01-24 14:37:41|         228|         195|
|2021-01-12 07:49:18|2021-01-12 08:11:06|         216|         196|
|2021-01-08 11:54:39|2021-01-08 12:09:38|          26|          26|
|2021-01-22 21:05:18|2021-01-22 21:30:05|         159|         265|
|2021-01-12 14:56:25|2021-01-12 15:14:23|          50|         145|
|2021-01-29 11:21:53|2021-01-29 11:31:07|         236|         162|
|2021-01-31 08:13:26|2021-01-31 08:16:01|         168|         147|
|2021-01-04 18:05:26|2021-01-04 18:26:59|          85|          39|
|2021-01-16 10:07:19|2021-01-16 10:11:07|       

In [23]:
from pyspark.sql import functions as F

In [29]:
df\
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-30|  2021-01-30|         261|          50|
| 2021-01-14|  2021-01-14|           4|          88|
| 2021-01-30|  2021-01-30|          78|          81|
| 2021-01-16|  2021-01-16|         213|          58|
| 2021-01-23|  2021-01-23|         188|         188|
| 2021-01-04|  2021-01-04|          17|          17|
| 2021-01-24|  2021-01-24|         228|         195|
| 2021-01-12|  2021-01-12|         216|         196|
| 2021-01-08|  2021-01-08|          26|          26|
| 2021-01-02|  2021-01-02|          28|         130|
| 2021-01-22|  2021-01-22|         159|         265|
| 2021-01-12|  2021-01-12|          50|         145|
| 2021-01-28|  2021-01-28|         140|         141|
| 2021-01-29|  2021-01-29|         236|         162|
| 2021-01-31|  2021-01-31|         168|         147|
| 2021-01-14|  2021-01-14|          97|       